In [3]:
import pandas as pd
import numpy as np
import spotipy
from spotipy.oauth2 import SpotifyOAuth
import regex

from urllib.parse import urlsplit, parse_qs

from sklearn.preprocessing import LabelEncoder

import cred

pd.set_option('display.max_columns', None)

# Redesign

In [ ]:
# PUBLIC VARIABLES
SCOPE = 'playlist-modify-public'
PLAYLIST_URL = "https://open.spotify.com/playlist/5dLuHuD3li0RIvYfAT4Kt0?si=8609e4e02f8545f2"

In [4]:
def spotify_uri_from_url(
        playlist_url: str
) -> str:
    """Given a spotify playlist url, return the playlist's uri

    Args:
        playlist_url (str): the url of the playlist

    Returns:
        str: the uri of the playlist
    """
    url_parts = urlsplit

    # Get the playlist ID
    path_parts = url_parts.path.split('/')
    playlist_id = path_parts[-1]

    # Check for query params that we do not want
    query_params = parse_qs(url_parts.query)
    if 'si' in query_params:
        playlist_id = query_params['si'][0]

    return f'spotify:playlist:{playlist_id}'

In [ ]:
sp = spotipy.Spotify(
    auth_manager=SpotifyOAuth(
        client_id=cred.client_ID,
        client_secret=cred.client_SECRET,
        redirect_uri=cred.redirect_url,
        scope=SCOPE
    )
)

In [596]:
playlist = pd.read_csv('/Users/CurtisIrvine/Downloads/Birthday.csv', on_bad_lines='skip')

In [597]:
# Score the dates between 0 and 100 

# Order the data frame descending by album date
playlist = playlist.sort_values(by=['Album Date'], ascending=False)

# Encode the album date column
playlist['album_date_encoded'] = LabelEncoder().fit_transform(playlist['Album Date'])

# Normalise the album date encoded field between 0 and 100
playlist['album_date_encoded'] = playlist['album_date_encoded'] / playlist['album_date_encoded'].max() * 100

In [598]:
# Normalise the # field between 0 and 100
playlist['Intuition'] = playlist['#'] / playlist['#'].max() * 100

# Map 0 to 100 and 100 to 0 
playlist['Intuition'] = playlist['Intuition'].map(lambda x: 100 - x)

In [599]:
# Use regex to get rid of '-' and ' db' from the loudness column
playlist['Loudness'] = playlist['Loudness'].apply(lambda x: regex.sub(r'[-]|[db]', '', x))

In [600]:
# Set the type of loudness column to int
playlist['Loudness'] = playlist['Loudness'].astype(int)

# Normalise the loudness column between 0 and 100
playlist['Loudness'] = playlist['Loudness']/playlist['Loudness'].max() * 100

# Map 100 to 0 and 0 to 100
playlist['Loudness'] = playlist['Loudness'].map(lambda x: 100 - x)

In [601]:
# Convert Time field from object of MM:SS to seconds
playlist['Time'] = playlist['Time'].apply(lambda x: int(x.split(':')[0]) * 60 + int(x.split(':')[1]))

In [602]:
# Create a composite metric to order the data frame by
composite_columns = [
    'album_date_encoded',
    'Danceability',
    'Energy',
    'Intuition',
    'Intuition',
    # 'Intuition',
    'Loudness'
]

# Add together the values of composite_columns in the playlist dataframe and divide by the number of columns
playlist['composite_metric'] = playlist[composite_columns].sum(axis=1)/len(composite_columns)

In [603]:
# Order the data frame by the composite metric
playlist = playlist.sort_values(by=['composite_metric'], ascending=False)

In [604]:
playlist

,#,Song,Artist,Popularity,Genres,Parent Genres,Album,Album Date,Time,Danceability,Energy,Acousticness,Instrumentalness,Happiness,Speechiness,Liveness,Loudness,Tempo,Key,Time Signature,Added At,Spotify Track Id,album_date_encoded,Intuition,composite_metric
0,1,El Apagón,Bad Bunny,85,"latin, reggaeton, trap latino",Latin,Un Verano Sin Ti,2022-05-06,201,90,100,10,0,40,NaN,14,71.428571,176,NaN,NaN,2022-06-06,0UvZcEfpzVyx47QsRbjyBz,94.776119,99.534884,92.545743
3,4,Kill V. Maim,Grimes,64,"art pop, canadian electropop, dance pop, grave...","Pop, Dance/Electronic",Art Angels,2015-11-06,246,100,100,1,8,46,4.0,11,85.714286,134,B Minor,4.0,2022-06-06,3WXhshrs1fzwF3rQE399Gq,28.358209,98.139535,85.058594
1,2,Good PusS (feat. cupcakKe) - Remix,"COBRAH,cupcakKe",52,"alternative r&b, dance pop, electropop, escape...","R&B, Hip Hop, Pop",Good PusS (feat. cupcakKe) [Remix],2022-02-23,219,85,69,20,30,32,6.0,19,50.000000,125,F Major,4.0,2022-06-06,0jiThGoZ1qdJrImd4U6XE5,92.537313,99.069767,82.446141
10,11,Tití Me Preguntó,Bad Bunny,98,"latin, reggaeton, trap latino",Latin,Un Verano Sin Ti,2022-05-06,243,65,72,10,0,19,25.0,13,64.285714,107,F Minor,4.0,2022-06-06,1IHWl5LamUGEuP4ozKQSXZ,94.776119,94.883721,80.971546
52,53,MAMI,"Chris Lorenzo,COBRAH",63,"bass house, deep groove house, electro house, ...","R&B, Dance/Electronic, Pop",MAMI,2022-07-22,247,76,94,3,10,83,6.0,4,64.285714,126,D#/E♭ Minor,4.0,2022-08-17,6JjSoJ2laV4ZbKNb9nybvh,99.253731,75.348837,80.706187
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
197,198,Genesis,Grimes,65,"art pop, canadian electropop, dance pop, grave...","Pop, Dance/Electronic",Visions,2012-03-12,255,61,68,8,7,23,3.0,24,50.000000,166,A#/B♭ Minor,4.0,2022-08-17,0yljUudXzjVcGEoYmLB17X,17.164179,7.906977,35.329689
134,135,Cinnamon Girl,Lana Del Rey,74,"art pop, pop",Pop,Norman Fucking Rockwell!,2019-08-30,300,29,34,81,41,14,4.0,14,21.428571,92,F#/G♭ Minor,5.0,2022-08-17,2mdEsXPu8ZmkHRRtAdC09e,50.000000,37.209302,34.807863
211,212,"New Person, Same Old Mistakes",Tame Impala,78,"australian psych, neo-psychedelic","Metal, Folk/Acoustic",Currents,2015-07-17,363,47,81,27,1,49,5.0,5,50.000000,152,G#/A♭ Major,4.0,2022-08-17,52ojopYMUzeNcudsoz7O9D,27.611940,1.395349,34.733773
151,152,Season Of The Witch,Lana Del Rey,60,"art pop, pop",Pop,"Season Of The Witch (From The Motion Picture ""...",2019-08-09,247,67,21,90,3,36,4.0,11,0.000000,96,D Major,4.0,2022-08-17,5DSi7heBC8eTIFROBvttnp,49.253731,29.302326,32.643064


In [583]:
# Number of songs considered peak
peak_number = 40

# Store the last 150 entries in a second dataframe
low_songs = playlist.iloc[-(len(playlist) - peak_number):]

# Drop these from playlist
playlist = playlist.drop(low_songs.index)

# Sort playlist by composite metric ascending
playlist = playlist.sort_values(by=['composite_metric'], ascending=True)

# Randomly select songs from the low_songs dataframe
first_songs = low_songs.sample(n=50)

# Order first_songs by the composite metric
first_songs = first_songs.sort_values(by=['composite_metric'], ascending=True)

# Drop these from low_songs
final_songs = low_songs.drop(first_songs.index)

# Order final_songs by the composite metric
final_songs = final_songs.sort_values(by=['composite_metric'], ascending=False)

In [584]:
# Find time of first songs
ramp_up = first_songs['Time'].sum()

# Return as hours 
print('Ramp up time is {} hours'.format(ramp_up / 3600))

Ramp up time is 3.120833333333333 hours


In [585]:
# Concatenate first_songs to the start of playlist
playlist = pd.concat([first_songs, playlist])

# Concatenate final_songs to the end of playlist
playlist = pd.concat([playlist, final_songs])

In [586]:
playlist

,#,Song,Artist,Popularity,Genres,Parent Genres,Album,Album Date,Time,Danceability,Energy,Acousticness,Instrumentalness,Happiness,Speechiness,Liveness,Loudness,Tempo,Key,Time Signature,Added At,Spotify Track Id,album_date_encoded,Intuition,composite_metric
211,212,"New Person, Same Old Mistakes",Tame Impala,78,"australian psych, neo-psychedelic","Metal, Folk/Acoustic",Currents,2015-07-17,363,47,81,27,1,49,5.0,5,50.000000,152,G#/A♭ Major,4.0,2022-08-17,52ojopYMUzeNcudsoz7O9D,27.611940,1.395349,34.733773
175,176,bad guy,Billie Eilish,83,"art pop, electropop, pop",Pop,"WHEN WE ALL FALL ASLEEP, WHERE DO WE GO?",2019-03-29,194,70,43,33,13,56,38.0,10,21.428571,135,G Major,4.0,2022-08-17,2Fxmhks0bxGSBdJ92vM42m,45.522388,18.139535,36.038338
190,191,These Days,Wallows,66,"indie pop, modern rock, pop","Rock, Pop",Spring EP,2018-04-06,202,66,67,4,3,87,3.0,8,50.000000,93,C Major,4.0,2022-08-17,4gZDCOFuLCxCRWNhSEcOtJ,36.567164,11.162791,40.315458
188,189,Am I A Girl?,Poppy,53,"alt z, art pop, dance pop, electropop, escape ...","Pop, Hip Hop",Am I A Girl?,2018-10-31,217,66,67,1,0,46,4.0,11,50.000000,122,D Minor,4.0,2022-08-17,6v3b2lQADc1ugsXvrqYZIB,43.283582,12.093023,41.744938
193,194,Судно (Борис Рижий),Molchat Doma,74,"belarusian indie, russian post-punk, sovietwave","Folk/Acoustic, Rock, Dance/Electronic",Этажи,2018-09-07,141,48,75,70,86,47,7.0,11,71.428571,160,C♯/D♭ Major,4.0,2022-08-17,1SHB1hp6267UK9bJQUxYvO,39.552239,9.767442,42.252616
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
168,169,o m e n,Kim Petras,0,"art pop, dance pop, electropop, pop, post-teen...",Pop,"TURN OFF THE LIGHT, VOL. 1",2018-10-01,99,67,48,21,94,50,3.0,10,21.428571,120,G#/A♭ Major,4.0,2022-08-17,7eLnzmQ2OdnYLTV0hoadRI,40.298507,21.395349,36.586296
197,198,Genesis,Grimes,65,"art pop, canadian electropop, dance pop, grave...","Pop, Dance/Electronic",Visions,2012-03-12,255,61,68,8,7,23,3.0,24,50.000000,166,A#/B♭ Minor,4.0,2022-08-17,0yljUudXzjVcGEoYmLB17X,17.164179,7.906977,35.329689
134,135,Cinnamon Girl,Lana Del Rey,74,"art pop, pop",Pop,Norman Fucking Rockwell!,2019-08-30,300,29,34,81,41,14,4.0,14,21.428571,92,F#/G♭ Minor,5.0,2022-08-17,2mdEsXPu8ZmkHRRtAdC09e,50.000000,37.209302,34.807863
151,152,Season Of The Witch,Lana Del Rey,60,"art pop, pop",Pop,"Season Of The Witch (From The Motion Picture ""...",2019-08-09,247,67,21,90,3,36,4.0,11,0.000000,96,D Major,4.0,2022-08-17,5DSi7heBC8eTIFROBvttnp,49.253731,29.302326,32.643064


In [587]:
# Reset the index of the playlist dataframe
playlist = playlist.reset_index()

# Find the index of the element with the highest composite metric
best_song_position = playlist['composite_metric'].idxmax()

# Sum the time up to the best song position
peak_time = playlist['Time'].iloc[:best_song_position].sum()

# Print the peak time as hours
print("The peak time is {} hours".format(peak_time / 3600))

The peak time is 5.825277777777778 hours


# Spotipy

In [588]:
# Scope to modify the playlist
scope = 'playlist-modify-public'

# Authenticate
sp = spotipy.Spotify(
    auth_manager=SpotifyOAuth(
        client_id=cred.client_ID,
        client_secret=cred.client_SECRET,
        redirect_uri=cred.redirect_url,
        scope=scope
    )
)

In [589]:
playlist_order = playlist['#'].tolist()

In [590]:
items = playlist['Spotify Track Id'].tolist()
# Split into 100s 
items1 = items[:100]
items2 = items[100:200]
items3 = items[200:]

In [591]:
# Wipe Playlist
replace = sp.playlist_replace_items(cred.test_id,
                                    items=[])

In [592]:
# Add items
add = sp.playlist_add_items(cred.test_id,
                            items = items1
)
add = sp.playlist_add_items(cred.test_id,
                            items = items2
)
add = sp.playlist_add_items(cred.test_id,
                            items = items3
)